# A script to reproduce the M2 solution

Action:
$$
S_{11} = \int d^{11}x \sqrt{-g} \left[R - \frac{1}{2}\frac{1}{4!}F_4^2\right]
$$

Metric ansatz:
$$
ds_{11}^2=H^a(-dx_0^2 + dx_1^2 + dx_2^2) + H^{-\frac{ad}{D-d-2}} (dy_0^2 + dy_1^2 + ... + dy_7^2)
$$

# 1. Compute Ricci tensor

parameters

In [1]:
D = 11
d = 3
HARMONIC_DIMENSIONS = D - d

set up coordinates

In [2]:
import sympy as sp
# coordinates
x = sp.symbols(f"x0:{d}", real=True)
y = sp.symbols(f"y0:{D - d}", real=True)
coords = (*x,*y)


# Parameters and symbols
apar = sp.symbols('a', real=True)
r_sym = sp.symbols('r', real=True, positive=True)

# expressions for radial coordinate r and r**2
r2 = sum(y_i**2 for y_i in y)
r = sp.sqrt(r2)

# harmonic function H, both as function of r, and as symbols
H = sp.Function('H')(r)
H_, H_p, H_pp = sp.symbols("H H' H''") # symbols for H and its derivatives, easier to work with once derivatives are taken and only manipulating algebraically

print(f"We are in {D} dimensions, with coordinates {coords}")
print(f"Radial coordinate r = {r}, r^2 = {r2}")
print(f"Harmonic function H is harmonic in {HARMONIC_DIMENSIONS} dimensions.")

We are in 11 dimensions, with coordinates (x0, x1, x2, y0, y1, y2, y3, y4, y5, y6, y7)
Radial coordinate r = sqrt(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2 + y6**2 + y7**2), r^2 = y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2 + y6**2 + y7**2
Harmonic function H is harmonic in 8 dimensions.


In [3]:
from helpers.einstein_helpers import minkowski_metric, euclidean_metric
# metric
g_MN = sp.zeros(D)
g_MN[0:d,0:d] = H**(apar) * minkowski_metric(d)
g_MN[d:D,d:D] = H**(-apar * sp.Rational(d, D -d -2)) * euclidean_metric(D - d)
g_MN = sp.simplify(g_MN)
g = sp.simplify(sp.sqrt(-(sp.det(g_MN))))
inv_g_MN = g_MN.inv()

In [4]:
from helpers.einstein_helpers import ricci_tensor
R_MN = ricci_tensor(g_MN, coords)

Computed Gamma^0_...,Computed Gamma^1_...,Computed Gamma^2_...,Computed Gamma^3_...,Computed Gamma^4_...,Computed Gamma^5_...,Computed Gamma^6_...,Computed Gamma^7_...,Computed Gamma^8_...,Computed Gamma^9_...,Computed Gamma^10_...,Computed R[0,...],Computed R[1,...],Computed R[2,...],Computed R[3,...],Computed R[4,...],Computed R[5,...],Computed R[6,...],Computed R[7,...],Computed R[8,...],Computed R[9,...],Computed R[10,...],

# 2. Simplifying relevant terms

From this point on no more derivatives shall be taken

In [5]:
def harmonic_derivative_subs(expr):
    # First sub in r and r^2
    expr = sp.simplify(expr)
    expr = sp.simplify(expr.subs({sp.sqrt(r2): r_sym,
                                  r2: r_sym**2}))
    
    # Then sub symbols of H, H', H'' for its literal derivatives
    expr = sp.simplify(expr.subs({H: H_,
                                  sp.Derivative(sp.Function('H')(r_sym), r_sym): H_p,
                                  sp.Derivative(sp.Function('H')(r_sym), (r_sym, 2)): H_pp}))
    
    # now impose harmonic condition to eliminate H''
    expr = expr.subs({H_pp: -(HARMONIC_DIMENSIONS -1)/r_sym * H_p})
    return sp.simplify(expr)

Obtain simplified expression for $R_{00}$

In [6]:
R_00 = harmonic_derivative_subs(R_MN[0,0])
R_00 = sp.factor(R_00).subs(r**2, r_sym**2)
R_00

-H'**2*a*H(r)**(3*a/2 - 2)/2

In [7]:
R_33 = harmonic_derivative_subs(R_MN[3,3])
R_33 = R_33.subs(sum([2 * yi**2 for yi in y]), 2 * r_sym**2)
R_33

-H'**2*a*(9*a*y0**2 + 2*r**2)/(8*r**2*H(r)**2)